<a href="https://colab.research.google.com/github/suksur/NLP/blob/main/Bert%2BbiLSTM_Manglish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install transformers==3.3.1

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
train1 = pd.read_csv('/content/drive/MyDrive/bert_sentiments/train_data.csv')
le = LabelEncoder()
train1['labels'] = le.fit_transform(train1['sentiments'])
#train=train.drop(columns='label')
train1

,text,sentiments,labels
0,Athanu ivdethee oruu rithiii so poo mone dines...,negative,2
1,Appo kaannunavane appa ennu villikan pc ye kay...,negative,2
2,Poda naayinde mone...,negative,2
3,NepopullakajarivalenakandupadikeSarnumpranamam,negative,2
4,Samayathinu anusarich kolam maraavunna oru naa...,negative,2
...,...,...,...
19113,Mammokka Ennum oru lahariyaanu.Annum Ennum ath...,Positive,1
19114,Acting ഒരു രക്ഷയും illa.... baki illathoke pinne,Positive,1
19115,Ivide Palakkad Jayettan Fans club nnu ashamsak...,Positive,1
19116,padam polikkum HBD mammokka By Die hard ...,Positive,1


In [3]:
train=pd.DataFrame(columns=['labels','tweets'])
train['labels']=train1['labels']
train['tweets']=train1['text']
train

,labels,tweets
0,2,Athanu ivdethee oruu rithiii so poo mone dines...
1,2,Appo kaannunavane appa ennu villikan pc ye kay...
2,2,Poda naayinde mone...
3,2,NepopullakajarivalenakandupadikeSarnumpranamam
4,2,Samayathinu anusarich kolam maraavunna oru naa...
...,...,...
19113,1,Mammokka Ennum oru lahariyaanu.Annum Ennum ath...
19114,1,Acting ഒരു രക്ഷയും illa.... baki illathoke pinne
19115,1,Ivide Palakkad Jayettan Fans club nnu ashamsak...
19116,1,padam polikkum HBD mammokka By Die hard ...


In [4]:
train=train.dropna()

In [5]:
val=pd.read_csv('/content/drive/MyDrive/bert_sentiments/validation_data.csv')
print(val)
#val['tweet']=val['text']
#val=val.drop(columns=['Unnamed: 2','text'])
val = val.dropna()
val['labels']=le.transform(val['sentiments'])
val['tweets']=val['text']
val=val.drop(columns=['sentiments','text'])
val

                                                 text sentiments
0   Epozhum challenges cheyumbo entho oru. Boring....   negative
1     Vello paniyum cheythu jeevukku. Childish people   negative
2    Background bynkra noise.. But the game is good♥️   negative
3   Entho oru Voice bhayangara disturbance aakkuva...   negative
4                              Clear illallooo chechy   negative
..                                                ...        ...
82                                Lijo oru shudhana .   Positive
83                                Allaavarumm polii🥰💕   Positive
84  State - Bangalore Lijo polichu.. Njan Karnatak...   Positive
85          Yee......nammadee pilleeer ethiiii🥳🥳🥳🥳🤙🤙🤙   Positive
86  Jeeva chetta, automobile channels best @Talkin...        NaN

[87 rows x 2 columns]


,labels,tweets
0,2,Epozhum challenges cheyumbo entho oru. Boring....
1,2,Vello paniyum cheythu jeevukku. Childish people
2,2,Background bynkra noise.. But the game is good♥️
3,2,Entho oru Voice bhayangara disturbance aakkuva...
4,2,Clear illallooo chechy
...,...,...
81,1,Nice game inim kondu varane... lijochettan😍he ...
82,1,Lijo oru shudhana .
83,1,Allaavarumm polii🥰💕
84,1,State - Bangalore Lijo polichu.. Njan Karnatak...


In [6]:

import pandas as pd
from torch.utils.data import Dataset,DataLoader

class RFDataset(Dataset):
  def __init__(self,text,label,tokenizer,max_len):
    self.text = text
    self.label = label
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item])
    label = self.label[item]

    encoding = self.tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )

    return {
        'text' : text,
        'input_ids' : encoding['input_ids'].flatten(),
        'attention_mask' : encoding['attention_mask'].flatten(),
        'label' : torch.tensor(label,dtype=torch.long)

    }

In [7]:
 
print('Training set size:',train.shape)
#Uncomment the next line when we have the test data
#print('Testing set size:',test.shape)
print('validation set size:',val.shape)

Training set size: (19116, 2)
validation set size: (86, 2)


In [8]:
import numpy as np

from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                                  classes = np.unique(train.labels.values),
                                                  y = train.labels.values)
class_weights

array([1.01837942, 0.96050648, 1.02361446])

In [9]:

def create_data_loader(df,tokenizer,max_len,batch_size):
  ds = RFDataset(
      text = df.tweets.to_numpy(),
      label = df.labels.to_numpy(),
      tokenizer = tokenizer,
      max_len = max_len
  )

  return DataLoader(ds,
                    batch_size = batch_size,
                    shuffle = True,
                    num_workers=2)
  

In [14]:
!pip install transformers==3.0.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.0-py3-none-any.whl (754 kB)
  Using cached sentencepiece-0.1.98-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.8.0rc4.tar.gz (96 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [13]:
from transformers import XLNetTokenizer,XLNetModel,AdamW,get_linear_schedule_with_warmup,AutoModel,AutoTokenizer
device = 'cuda'
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-uncased'
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


ModuleNotFoundError: ignored

In [ ]:
BATCH_SIZE = 35
MAX_LEN = 128
train_data_loader = create_data_loader(train,tokenizer,MAX_LEN,BATCH_SIZE)
val_data_loader = create_data_loader(val,tokenizer,MAX_LEN,BATCH_SIZE)

In [ ]:
BERT_model = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

In [ ]:
import torch
import torch.nn as nn
class RFClassifier(nn.Module):
  def __init__(self, n_classes):
    super(RFClassifier, self).__init__()
    self.auto = AutoModel.from_pretrained('bert-base-multilingual-uncased')
    self.lstm = nn.LSTM(768, 256, batch_first=True,bidirectional=True)
    self.linear = nn.Linear(256*2, 128)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.4)
    self.out = nn.Linear(128, n_classes)
  
  def forward(self, input_ids, attention_mask):
    sequence_output, pooled_output = self.auto(input_ids, 
               attention_mask=attention_mask)

    # sequence_output has the following shape: (batch_size, sequence_length, 768)
    lstm_output, (h,c) = self.lstm(sequence_output) ## extract the 1st token's embeddings
    hidden = torch.cat((lstm_output[:,-1, :256],lstm_output[:,0, 256:]),dim=-1)
    linear_output = self.linear(lstm_output[:,-1].view(-1,256*2)) ### assuming that you are only using the output of the last LSTM cell to perform classification

    return linear_output

In [ ]:
model = RFClassifier(5)
model = model.to(device)

RuntimeError: ignored

In [ ]:
EPOCHS = 27
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:

def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
            )
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs,labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["label"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# i = 0
# for data in train_data_loader:
#   if i < 0:
#     break
#   input_ids = data['input_ids'].to(device)
#   attention_mask = data['attention_mask'].to(device)
#   labels = data['label'].to(device)
#   # print(data['text'])
#   print(input_ids)
#   print(attention_mask)
#   outputs = model(
#             input_ids=input_ids,
#             attention_mask=attention_mask
#             )
#   print(outputs)
#   i -= 1


In [ ]:
from collections import defaultdict
import torch
 
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
 
 
  start_time = time.time()
  # print(type(train_data_loader))
  train_acc,train_loss = train_epoch(
      model = model,
      data_loader = train_data_loader,
      loss_fn = loss_fn,
      optimizer = optimizer,
      device = device,
      scheduler = scheduler,
      n_examples = len(train)
  )
   
  
  val_acc,val_loss = eval_model(
      model,
      val_data_loader,
      loss_fn,
      device,
      len(val)
  )
  
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'Train Loss {train_loss} accuracy {train_acc}')
  print(f'Val Loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(),'bert-base-multilingual-uncased.bin')
    best_accuracy = val_acc

In [ ]:
# history

In [ ]:
#type (history)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['train_acc'],label ='train accuracy')
plt.plot(history['val_acc'],label ='validation accuracy')
plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch') 
plt.legend() 
plt.ylim([0, 1]);  

In [ ]:
val_acc, _ = eval_model(
  model,
  val_data_loader,
  loss_fn,
  device,
  len(val) #Change it to test when you have the test results
)
val_acc.item()

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  sentence = []
  predictions = []
  prediction_probs = []
  real_values = []
  with torch.no_grad():
    for d in data_loader:
      texts = d["text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      labels = d["label"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      sentence.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(labels)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return sentence, predictions, prediction_probs, real_values

In [ ]:

y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  val_data_loader
)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test, y_pred,zero_division=0, digits=4))

In [ ]:
y_pred[0].detach().cpu()

In [ ]:
for i in range(len(y_review_texts)):
  print(y_review_texts[i], " : ", y_pred[i])
  # y_pred[i]